In [83]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import ipywidgets as W
from IPython.display import display, clear_output, HTML

# Dossiers projet
ROOT_DIR  = os.path.abspath(os.path.join(".."))
RAW_DIR   = os.path.join(ROOT_DIR, "data", "raw")
CLEAN_DIR = os.path.join(ROOT_DIR, "data", "clean")
Path(CLEAN_DIR).mkdir(parents=True, exist_ok=True)

# Import du kit UI (si tu as récupéré dvfkit.py)
import sys
sys.path.append("/mnt/data")  # adapte si besoin (ex: le placer à la racine projet)
import dvfkit


In [84]:
import sys, os, pandas as pd
sys.path.append("/mnt/data")
import dvfkit, realestate_tools as rt

# 1) Charger DVF propre
df = dvfkit.load_df(clean_dir=CLEAN_DIR)

# 2) (Optionnel) Charger loyers/gares si présents
loyers = pd.read_parquet(os.path.join(CLEAN_DIR, "loyers_idf.parquet")) if os.path.exists(os.path.join(CLEAN_DIR, "loyers_idf.parquet")) else None
gares  = pd.read_parquet(os.path.join(CLEAN_DIR, "gares_idf.parquet"))  if os.path.exists(os.path.join(CLEAN_DIR, "gares_idf.parquet"))  else None

# 3) Unifier
dfu = rt.unify_all(df, loyers, gares)

# 4) Démarrer dashboard (mêmes filtres pour tous les onglets)
boot = dvfkit.boot_dashboard(dfu, theme="light", topn=10,
                             title="Investissement locatif — Île-de-France",
                             subtitle="Dashboard unifié (DVF + loyers + gares)")
controls = dvfkit.get_controls(boot)


In [85]:
# ==== Chargement des données propres (provenant du pipeline DVF) ====
# On suppose que 'clean_fp' pointe vers le parquet consolidé (créé en amont).
df = pd.read_parquet(clean_fp).copy()

# Sécurisation colonnes essentielles
for c in ["prix_m2","surface_reelle_bati","valeur_fonciere","annee","nom_commune","code_postal"]:
    if c not in df.columns:
        df[c] = np.nan


In [86]:
def add_dual_widget_tab(boot, title):
    """Crée un onglet avec 2 sorties côte à côte (Widget A / Widget B)."""
    boxA = W.Output()
    boxB = W.Output()
    container = W.HBox(
        [W.VBox([boxA], layout=W.Layout(width="50%")), 
         W.VBox([boxB], layout=W.Layout(width="50%"))],
        layout=W.Layout(width="100%")
    )
    out_tab = dvfkit.add_tab(boot, title, output_widget=None)  # obtient un Output vide
    with out_tab:
        clear_output(wait=True)
        display(container)
    return boxA, boxB

def render_dual(boxA, boxB, fn_left, fn_right):
    """Rend les deux widgets via deux fonctions: fn_left(df_filt, controls), fn_right(...)."""
    with boxA:
        clear_output(wait=True)
        fn_left()
    with boxB:
        clear_output(wait=True)
        fn_right()


In [87]:
# ==== Filtres & calculs ====
def iqr_bounds(s, k=2.0):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    return q1 - k*iqr, q3 + k*iqr

def apply_filters(d):
    # surface
    smin, smax = w_surface.value
    d = d[d["surface_reelle_bati"].between(smin, smax)]
    # années
    if isinstance(w_year, W.SelectionRangeSlider):
        y0, y1 = w_year.value
        d = d[(d["annee"]>=y0) & (d["annee"]<=y1)]
    # commune
    if w_commune.value != "(Toutes)":
        d = d[d["nom_commune"] == w_commune.value]
    # outliers
    if w_iqr.value and len(d) >= 50 and d["prix_m2"].notna().any():
        lo, hi = iqr_bounds(d["prix_m2"], k=2.0)
        d = d[d["prix_m2"].between(lo, hi)]
    # yield brut
    loyer_m2 = w_loyer.value
    d = d.assign(revenu_annuel = loyer_m2 * d["surface_reelle_bati"] * 12.0)
    d = d.assign(yield_brut    = d["revenu_annuel"] / d["valeur_fonciere"])
    return d


In [88]:
outA_clean, outB_clean = add_dual_widget_tab(boot, "Nettoyage & Profil DVF")

def clean_widget_left():
    d = dvfkit.apply_filters(df, controls)
    # Profil simple: tailles, NA, bornes prix/m²
    n = len(d)
    na_prix = int(d["prix_m2"].isna().sum())
    p10, p50, p90 = (d["prix_m2"].quantile([.1,.5,.9]).round(0).astype("Int64")
                     if d["prix_m2"].notna().any() else (pd.Series([np.nan]*3)))
    html = f"""
    <div class="card">
      <b>Profil dataset filtré</b>
      <div class="note">Taille: {n:,} | prix/m² NA: {na_prix:,}</div>
      <div class="note">Déciles prix/m² (10/50/90): {p10:.0f} / {p50:.0f} / {p90:.0f} €</div>
    </div>
    """.replace(",", " ")
    display(HTML(html))

    # Histogramme surface (Matplotlib)
    if n:
        import matplotlib.pyplot as plt
        s = d["surface_reelle_bati"].dropna()
        if len(s):
            plt.figure(figsize=(6.5,3.8))
            plt.hist(s, bins=40)
            plt.title("Distribution des surfaces (m²)")
            plt.xlabel("Surface (m²)"); plt.ylabel("Fréquence")
            plt.grid(alpha=.2); plt.tight_layout(); plt.show()

def clean_widget_right():
    d = dvfkit.apply_filters(df, controls)
    if len(d):
        # Scatter prix/m² vs surface (Plotly si dispo, sinon Matplotlib)
        try:
            import plotly.express as px
            ds = d.sample(min(4000, len(d)), random_state=0)
            fig = px.scatter(ds, x="surface_reelle_bati", y="prix_m2",
                             hover_data=["nom_commune","code_postal","annee"],
                             template="plotly_white", height=400, trendline="lowess")
            fig.update_xaxes(title="Surface (m²)"); fig.update_yaxes(title="Prix/m² (€)")
            fig.show()
        except Exception:
            import matplotlib.pyplot as plt
            ds = d.sample(min(4000, len(d)), random_state=0)
            plt.figure(figsize=(6.5,3.8))
            plt.scatter(ds["surface_reelle_bati"], ds["prix_m2"], s=12, alpha=.6)
            plt.title("Prix/m² vs Surface (échantillon)")
            plt.xlabel("Surface (m²)"); plt.ylabel("Prix/m² (€)")
            plt.grid(alpha=.2); plt.tight_layout(); plt.show()

render_dual(outA_clean, outB_clean, clean_widget_left, clean_widget_right)


In [89]:
# ==== Mise en page (onglets + barre de presets) & Observers ====
# Barre presets
bar_presets = W.HBox([btn_studio, btn_t2, btn_t3], layout=W.Layout(justify_content="flex-start"))

# Panneau de paramètres
controls_panel = W.VBox([
    W.HTML('<div class="card"><b>Paramètres d’analyse</b></div>'),
    W.VBox([
        bar_presets,
        w_surface, w_loyer, w_topn, w_iqr, w_hist, w_year, w_commune,
        W.HBox([w_btn_export, w_msg])
    ], layout=W.Layout(padding="0 6px 8px 6px"))
])

# Onglets de vues
tabs = W.Tab(children=[
    W.VBox([out_overview]),
    W.VBox([out_table]),
    W.VBox([out_plot1]),
    W.VBox([out_plot2]),
])
tabs.set_title(0, "Vue d’ensemble")
tabs.set_title(1, "Communes")
tabs.set_title(2, "Dispersion")
tabs.set_title(3, "Distribution")

# Layout global
display(W.HBox([controls_panel, tabs], layout=W.Layout(width="100%"),))

# Observers
for w in (w_surface, w_loyer, w_topn, w_iqr, w_hist, w_commune):
    w.observe(render, "value")
if isinstance(w_year, W.SelectionRangeSlider):
    w_year.observe(render, "value")
w_btn_export.on_click(on_export_clicked)

# Premier rendu
render()


In [90]:
import ipywidgets as W
from IPython.display import display, clear_output, HTML
import numpy as np
import pandas as pd

# ---------- Helpers UI ----------
def add_quad_widget_tab(boot, title):
    """Crée un onglet avec 4 sorties (2 x 2). Retourne (w1, w2, w3, w4)."""
    tab_root = W.Output()
    container = dvfkit.add_tab(boot, title, output_widget=tab_root)

    w1 = W.Output(); w2 = W.Output(); w3 = W.Output(); w4 = W.Output()
    col_left  = W.VBox([w1, w3], layout=W.Layout(width="50%", padding="0 8px 0 0"))
    col_right = W.VBox([w2, w4], layout=W.Layout(width="50%", padding="0 0 0 8px"))
    grid = W.HBox([col_left, col_right], layout=W.Layout(width="100%"))

    with tab_root:
        clear_output(wait=True)
        display(grid)
    return w1, w2, w3, w4

def safe_plot_scatter(df_xy, x, y, xlabel, ylabel, hover=None, height=380):
    try:
        import plotly.express as px
        fig = px.scatter(df_xy, x=x, y=y, hover_data=hover or [],
                         template="plotly_white", height=height, opacity=0.85)
        fig.update_xaxes(title=xlabel); fig.update_yaxes(title=ylabel)
        fig.show()
    except Exception:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(6.5,3.8))
        plt.scatter(df_xy[x], df_xy[y], s=12, alpha=.7)
        plt.xlabel(xlabel); plt.ylabel(ylabel)
        plt.grid(alpha=.2); plt.tight_layout(); plt.show()

def safe_plot_hist(series, bins=45, xlabel="", ylabel="Fréquence", height=360, title=""):
    try:
        import plotly.express as px
        fig = px.histogram(series.dropna(), x=series.dropna(), nbins=bins,
                           template="plotly_white", height=height)
        fig.update_xaxes(title=xlabel); fig.update_yaxes(title=ylabel)
        if title: fig.update_layout(title=title)
        fig.show()
    except Exception:
        import matplotlib.pyplot as plt
        s = series.dropna()
        plt.figure(figsize=(6.5,3.8))
        plt.hist(s, bins=bins)
        if title: plt.title(title)
        plt.xlabel(xlabel); plt.ylabel(ylabel)
        plt.grid(alpha=.2); plt.tight_layout(); plt.show()

# ---------- 4 widgets : définitions ----------
def widget1_profil_distribution(dfu, controls):
    """Profil rapide + histogramme surfaces."""
    d = dvfkit.apply_filters(dfu, controls)
    n = len(d)
    na_prix = int(d["prix_m2"].isna().sum()) if "prix_m2" in d else 0
    if "prix_m2" in d and d["prix_m2"].notna().any():
        q = d["prix_m2"].quantile([.1,.5,.9]).round(0)
        p10, p50, p90 = int(q.iloc[0]), int(q.iloc[1]), int(q.iloc[2])
    else:
        p10=p50=p90=np.nan

    html = f"""
    <div class="card">
      <b>Profil dataset filtré</b>
      <div class="note">Taille: {n:,} | prix/m² NA: {na_prix:,}</div>
      <div class="note">Déciles prix/m² (10/50/90) : {p10} / {p50} / {p90} €</div>
    </div>
    """.replace(",", " ")
    display(HTML(html))

    if n and "surface_reelle_bati" in d and d["surface_reelle_bati"].notna().any():
        safe_plot_hist(d["surface_reelle_bati"], bins=40, xlabel="Surface (m²)",
                       title="Distribution des surfaces (m²)")

def widget2_dispersion_prix_surface(dfu, controls):
    """Dispersion prix/m² vs surface."""
    d = dvfkit.apply_filters(dfu, controls)
    if len(d) and {"surface_reelle_bati","prix_m2"}.issubset(d.columns):
        ds = d.dropna(subset=["surface_reelle_bati","prix_m2"])
        ds = ds.sample(min(4000, len(ds)), random_state=0) if len(ds)>4000 else ds
        if len(ds):
            safe_plot_scatter(ds, x="surface_reelle_bati", y="prix_m2",
                              xlabel="Surface (m²)", ylabel="Prix/m² (€)",
                              hover=["nom_commune","code_postal","annee"], height=420)
        else:
            display(HTML('<div class="card">Aucune donnée exploitable pour ce nuage.</div>'))
    else:
        display(HTML('<div class="card">Colonnes requises absentes.</div>'))

def widget3_comparatif_prix_loyer(dfu, controls):
    """Comparatif prix/m² médian vs loyer/m² médian (si loyer_m2 présent)."""
    d = dvfkit.apply_filters(dfu, controls)
    if "loyer_m2" not in d.columns:
        display(HTML('<div class="card">Dataset loyers non attaché (colonne loyer_m2 absente).</div>'))
        return
    j = d.dropna(subset=["prix_m2","loyer_m2"])
    if len(j):
        g = (j.groupby(["nom_commune","code_postal"], as_index=False)
               .agg(prix_m2_med=("prix_m2","median"),
                    loyer_m2_med=("loyer_m2","median")))
        g = g.dropna(subset=["prix_m2_med","loyer_m2_med"])
        if len(g):
            safe_plot_scatter(g, x="prix_m2_med", y="loyer_m2_med",
                              xlabel="Prix/m² médian (€)", ylabel="Loyer/m² médian (€)",
                              hover=["nom_commune","code_postal"])
        else:
            display(HTML('<div class="card">Données loyers insuffisantes après agrégation.</div>'))
    else:
        display(HTML('<div class="card">Aucune ligne avec prix_m2 et loyer_m2.</div>'))

def widget4_gare_vs_rendement(dfu, controls):
    """Temps jusqu’à gare vs rendement brut (si colonnes présentes)."""
    d = dvfkit.apply_filters(dfu, controls)
    needed = {"temps_gare_min","yield_brut"}
    if not needed.issubset(d.columns):
        msg = " / ".join(sorted(needed - set(d.columns)))
        display(HTML(f'<div class="card">Colonnes manquantes pour ce widget : {msg}</div>'))
        return
    jj = d.dropna(subset=["temps_gare_min","yield_brut"])
    if len(jj):
        tmp = jj.copy()
        tmp["yield_%"] = tmp["yield_brut"]*100
        safe_plot_scatter(tmp, x="temps_gare_min", y="yield_%",
                          xlabel="Temps jusqu’à gare (min)", ylabel="Rendement brut (%)",
                          hover=["nom_commune","code_postal","annee"])
    else:
        display(HTML('<div class="card">Aucune donnée exploitable (temps_gare_min & yield_brut).</div>'))

# ---------- Création de l’onglet + rendu initial ----------
w1, w2, w3, w4 = add_quad_widget_tab(boot, "Widgets unifiés (x4)")

def render_all_widgets(*_):
    with w1:
        clear_output(wait=True); widget1_profil_distribution(dfu, controls)
    with w2:
        clear_output(wait=True); widget2_dispersion_prix_surface(dfu, controls)
    with w3:
        clear_output(wait=True); widget3_comparatif_prix_loyer(dfu, controls)
    with w4:
        clear_output(wait=True); widget4_gare_vs_rendement(dfu, controls)

# Observers : re-render quand filtres changent
for key in ("w_surface","w_loyer","w_topn","w_iqr","w_hist","w_commune"):
    if key in controls:
        controls[key].observe(render_all_widgets, "value")
if isinstance(controls.get("w_year"), W.SelectionRangeSlider):
    controls["w_year"].observe(render_all_widgets, "value")

# Rendu initial
render_all_widgets()


In [91]:
# Exemple: charge un parquet externe si disponible (sinon None)
loyers_path = os.path.join(CLEAN_DIR, "loyers_idf.parquet")
loyers_df = pd.read_parquet(loyers_path) if os.path.exists(loyers_path) else None

outA_loyers, outB_loyers = add_dual_widget_tab(boot, "Loyers IDF")

def loyers_widget_left():
    d = dvfkit.apply_filters(df, controls)
    if loyers_df is not None and "code_postal" in loyers_df.columns:
        j = d.merge(loyers_df, on="code_postal", how="left", suffixes=("", "_loyer"))
        # Supposons colonne "loyer_m2"
        if "loyer_m2" in j.columns:
            # Comparatif prix_m2 vs loyer_m2 médian par commune
            g = (j.groupby(["nom_commune","code_postal"], as_index=False)
                   .agg(prix_m2_med=("prix_m2","median"),
                        loyer_m2_med=("loyer_m2","median")))
            g = g.dropna(subset=["prix_m2_med","loyer_m2_med"])
            try:
                import plotly.express as px
                fig = px.scatter(g, x="prix_m2_med", y="loyer_m2_med",
                                 hover_data=["nom_commune","code_postal"],
                                 template="plotly_white", height=400)
                fig.update_xaxes(title="Prix/m² médian (€)"); fig.update_yaxes(title="Loyer/m² médian (€)")
                fig.show()
            except Exception:
                import matplotlib.pyplot as plt
                plt.figure(figsize=(6.5,3.8))
                plt.scatter(g["prix_m2_med"], g["loyer_m2_med"], s=14, alpha=.7)
                plt.xlabel("Prix/m² médian (€)"); plt.ylabel("Loyer/m² médian (€)")
                plt.grid(alpha=.2); plt.tight_layout(); plt.show()
        else:
            display(HTML('<div class="card">⚠️ loyers_df chargé mais sans colonne "loyer_m2".</div>'))
    else:
        display(HTML('<div class="card">Aucun dataset loyers externe trouvé — widget comparatif non rendu.</div>'))

def loyers_widget_right():
    d = dvfkit.apply_filters(df, controls)
    # Distribution du rendement brut (déjà calculé via controls/loyer slider)
    if len(d) and d["yield_brut"].notna().any():
        try:
            import plotly.express as px
            y = (d["yield_brut"]*100).clip(upper=(d["yield_brut"]*100).quantile(0.99))
            fig = px.histogram(y, x=y, nbins=45, template="plotly_white", height=380)
            fig.update_xaxes(title="Rendement brut (%)"); fig.update_yaxes(title="Fréquence")
            fig.show()
        except Exception:
            import matplotlib.pyplot as plt
            y = (d["yield_brut"]*100).dropna()
            y = y.clip(upper=y.quantile(0.99))
            plt.figure(figsize=(6.5,3.8))
            plt.hist(y, bins=45)
            plt.xlabel("Rendement brut (%)"); plt.ylabel("Fréquence")
            plt.grid(alpha=.2); plt.tight_layout(); plt.show()

render_dual(outA_loyers, outB_loyers, loyers_widget_left, loyers_widget_right)


In [92]:
gares_path = os.path.join(CLEAN_DIR, "gares_idf.parquet")
gares_df = pd.read_parquet(gares_path) if os.path.exists(gares_path) else None

outA_gare, outB_gare = add_dual_widget_tab(boot, "Accessibilité gares")

def gare_widget_left():
    d = dvfkit.apply_filters(df, controls)
    if gares_df is not None and "code_postal" in gares_df.columns:
        j = d.merge(gares_df, on="code_postal", how="left")
        # Hypothèse de colonnes: temps_gare_min (float, plus petit = mieux)
        if "temps_gare_min" in j.columns:
            # Scatter temps_gare vs rendement
            jj = j.dropna(subset=["temps_gare_min","yield_brut"])
            if len(jj):
                try:
                    import plotly.express as px
                    fig = px.scatter(jj, x="temps_gare_min", y=(jj["yield_brut"]*100),
                                     hover_data=["nom_commune","code_postal","annee"],
                                     template="plotly_white", height=400)
                    fig.update_xaxes(title="Temps jusqu’à gare (min)"); fig.update_yaxes(title="Rendement brut (%)")
                    fig.show()
                except Exception:
                    import matplotlib.pyplot as plt
                    plt.figure(figsize=(6.5,3.8))
                    plt.scatter(jj["temps_gare_min"], jj["yield_brut"]*100, s=12, alpha=.6)
                    plt.xlabel("Temps jusqu’à gare (min)"); plt.ylabel("Rendement brut (%)")
                    plt.grid(alpha=.2); plt.tight_layout(); plt.show()
            else:
                display(HTML('<div class="card">Données insuffisantes pour le scatter.</div>'))
        else:
            display(HTML('<div class="card">⚠️ gares_df chargé mais sans colonne "temps_gare_min".</div>'))
    else:
        display(HTML('<div class="card">Aucun dataset gares externe trouvé — widget non rendu.</div>'))

def gare_widget_right():
    d = dvfkit.apply_filters(df, controls)
    if gares_df is not None and "code_postal" in gares_df.columns:
        j = d.merge(gares_df, on="code_postal", how="left")
        # Hypothèse: nb_gares_15min
        if "nb_gares_15min" in j.columns:
            g = (j.groupby(["nom_commune","code_postal"], as_index=False)["nb_gares_15min"]
                   .median().sort_values("nb_gares_15min", ascending=False).head(15))
            html = g.rename(columns={"nom_commune":"Commune","code_postal":"CP","nb_gares_15min":"Gares <=15min (méd.)"}) \
                    .style.hide(axis="index").to_html()
            display(HTML(f'<div class="card"><b>Top accessibilité (<= 15min gare)</b><div class="note">Médiane par commune</div>{html}</div>'))
        else:
            display(HTML('<div class="card">⚠️ gares_df sans colonne "nb_gares_15min".</div>'))
    else:
        display(HTML('<div class="card">Aucun dataset gares externe trouvé — widget non rendu.</div>'))

render_dual(outA_gare, outB_gare, gare_widget_left, gare_widget_right)


In [93]:
# Remplacer vos définitions par celles-ci

import ipywidgets as W
from IPython.display import display, clear_output, HTML

def add_dual_widget_tab(boot, title):
    """Crée un onglet avec 2 sorties côte à côte (Widget A / Widget B)."""
    # conteneur principal de l'onglet
    tab_container = W.Output()
    out_tab = dvfkit.add_tab(boot, title, output_widget=tab_container)

    # deux sorties enfant
    boxA = W.Output()
    boxB = W.Output()
    grid = W.HBox(
        [W.VBox([boxA], layout=W.Layout(width="50%")),
         W.VBox([boxB], layout=W.Layout(width="50%"))],
        layout=W.Layout(width="100%")
    )

    with tab_container:
        clear_output(wait=True)
        display(grid)

    return boxA, boxB

def render_dual(boxA, boxB, fn_left, fn_right):
    with boxA:
        clear_output(wait=True)
        fn_left()
    with boxB:
        clear_output(wait=True)
        fn_right()
from IPython.display import display, HTML
display(HTML('<div class="card">Rendu final relancé.</div>'))



In [94]:
# Si tu modifies des fonctions au-dessus, tu peux rappeler les render_dual:
render_dual(outA_clean, outB_clean, clean_widget_left, clean_widget_right)
render_dual(outA_loyers, outB_loyers, loyers_widget_left, loyers_widget_right)
render_dual(outA_gare, outB_gare, gare_widget_left, gare_widget_right)


In [95]:
import ipywidgets as W
from IPython.display import display, clear_output, HTML
import numpy as np
import pandas as pd

# ---------- Helpers UI ----------
def add_quad_widget_tab(boot, title):
    """Crée un onglet avec 4 sorties (2 x 2). Retourne (w1, w2, w3, w4)."""
    tab_root = W.Output()
    container = dvfkit.add_tab(boot, title, output_widget=tab_root)

    w1 = W.Output(); w2 = W.Output(); w3 = W.Output(); w4 = W.Output()
    col_left  = W.VBox([w1, w3], layout=W.Layout(width="50%", padding="0 8px 0 0"))
    col_right = W.VBox([w2, w4], layout=W.Layout(width="50%", padding="0 0 0 8px"))
    grid = W.HBox([col_left, col_right], layout=W.Layout(width="100%"))

    with tab_root:
        clear_output(wait=True)
        display(grid)
    return w1, w2, w3, w4

def safe_plot_scatter(df_xy, x, y, xlabel, ylabel, hover=None, height=380):
    try:
        import plotly.express as px
        fig = px.scatter(df_xy, x=x, y=y, hover_data=hover or [],
                         template="plotly_white", height=height, opacity=0.85)
        fig.update_xaxes(title=xlabel); fig.update_yaxes(title=ylabel)
        fig.show()
    except Exception:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(6.5,3.8))
        plt.scatter(df_xy[x], df_xy[y], s=12, alpha=.7)
        plt.xlabel(xlabel); plt.ylabel(ylabel)
        plt.grid(alpha=.2); plt.tight_layout(); plt.show()

def safe_plot_hist(series, bins=45, xlabel="", ylabel="Fréquence", height=360, title=""):
    try:
        import plotly.express as px
        fig = px.histogram(series.dropna(), x=series.dropna(), nbins=bins,
                           template="plotly_white", height=height)
        fig.update_xaxes(title=xlabel); fig.update_yaxes(title=ylabel)
        if title: fig.update_layout(title=title)
        fig.show()
    except Exception:
        import matplotlib.pyplot as plt
        s = series.dropna()
        plt.figure(figsize=(6.5,3.8))
        plt.hist(s, bins=bins)
        if title: plt.title(title)
        plt.xlabel(xlabel); plt.ylabel(ylabel)
        plt.grid(alpha=.2); plt.tight_layout(); plt.show()

# ---------- 4 widgets : définitions ----------
def widget1_profil_distribution(dfu, controls):
    """Profil rapide + histogramme surfaces."""
    d = dvfkit.apply_filters(dfu, controls)
    n = len(d)
    na_prix = int(d["prix_m2"].isna().sum()) if "prix_m2" in d else 0
    if "prix_m2" in d and d["prix_m2"].notna().any():
        q = d["prix_m2"].quantile([.1,.5,.9]).round(0)
        p10, p50, p90 = int(q.iloc[0]), int(q.iloc[1]), int(q.iloc[2])
    else:
        p10=p50=p90=np.nan

    html = f"""
    <div class="card">
      <b>Profil dataset filtré</b>
      <div class="note">Taille: {n:,} | prix/m² NA: {na_prix:,}</div>
      <div class="note">Déciles prix/m² (10/50/90) : {p10} / {p50} / {p90} €</div>
    </div>
    """.replace(",", " ")
    display(HTML(html))

    if n and "surface_reelle_bati" in d and d["surface_reelle_bati"].notna().any():
        safe_plot_hist(d["surface_reelle_bati"], bins=40, xlabel="Surface (m²)",
                       title="Distribution des surfaces (m²)")

def widget2_dispersion_prix_surface(dfu, controls):
    """Dispersion prix/m² vs surface."""
    d = dvfkit.apply_filters(dfu, controls)
    if len(d) and {"surface_reelle_bati","prix_m2"}.issubset(d.columns):
        ds = d.dropna(subset=["surface_reelle_bati","prix_m2"])
        ds = ds.sample(min(4000, len(ds)), random_state=0) if len(ds)>4000 else ds
        if len(ds):
            safe_plot_scatter(ds, x="surface_reelle_bati", y="prix_m2",
                              xlabel="Surface (m²)", ylabel="Prix/m² (€)",
                              hover=["nom_commune","code_postal","annee"], height=420)
        else:
            display(HTML('<div class="card">Aucune donnée exploitable pour ce nuage.</div>'))
    else:
        display(HTML('<div class="card">Colonnes requises absentes.</div>'))

def widget3_comparatif_prix_loyer(dfu, controls):
    """Comparatif prix/m² médian vs loyer/m² médian (si loyer_m2 présent)."""
    d = dvfkit.apply_filters(dfu, controls)
    if "loyer_m2" not in d.columns:
        display(HTML('<div class="card">Dataset loyers non attaché (colonne loyer_m2 absente).</div>'))
        return
    j = d.dropna(subset=["prix_m2","loyer_m2"])
    if len(j):
        g = (j.groupby(["nom_commune","code_postal"], as_index=False)
               .agg(prix_m2_med=("prix_m2","median"),
                    loyer_m2_med=("loyer_m2","median")))
        g = g.dropna(subset=["prix_m2_med","loyer_m2_med"])
        if len(g):
            safe_plot_scatter(g, x="prix_m2_med", y="loyer_m2_med",
                              xlabel="Prix/m² médian (€)", ylabel="Loyer/m² médian (€)",
                              hover=["nom_commune","code_postal"])
        else:
            display(HTML('<div class="card">Données loyers insuffisantes après agrégation.</div>'))
    else:
        display(HTML('<div class="card">Aucune ligne avec prix_m2 et loyer_m2.</div>'))

def widget4_gare_vs_rendement(dfu, controls):
    """Temps jusqu’à gare vs rendement brut (si colonnes présentes)."""
    d = dvfkit.apply_filters(dfu, controls)
    needed = {"temps_gare_min","yield_brut"}
    if not needed.issubset(d.columns):
        msg = " / ".join(sorted(needed - set(d.columns)))
        display(HTML(f'<div class="card">Colonnes manquantes pour ce widget : {msg}</div>'))
        return
    jj = d.dropna(subset=["temps_gare_min","yield_brut"])
    if len(jj):
        tmp = jj.copy()
        tmp["yield_%"] = tmp["yield_brut"]*100
        safe_plot_scatter(tmp, x="temps_gare_min", y="yield_%",
                          xlabel="Temps jusqu’à gare (min)", ylabel="Rendement brut (%)",
                          hover=["nom_commune","code_postal","annee"])
    else:
        display(HTML('<div class="card">Aucune donnée exploitable (temps_gare_min & yield_brut).</div>'))

# ---------- Création de l’onglet + rendu initial ----------
w1, w2, w3, w4 = add_quad_widget_tab(boot, "Widgets unifiés (x4)")

def render_all_widgets(*_):
    with w1:
        clear_output(wait=True); widget1_profil_distribution(dfu, controls)
    with w2:
        clear_output(wait=True); widget2_dispersion_prix_surface(dfu, controls)
    with w3:
        clear_output(wait=True); widget3_comparatif_prix_loyer(dfu, controls)
    with w4:
        clear_output(wait=True); widget4_gare_vs_rendement(dfu, controls)

# Observers : re-render quand filtres changent
for key in ("w_surface","w_loyer","w_topn","w_iqr","w_hist","w_commune"):
    if key in controls:
        controls[key].observe(render_all_widgets, "value")
if isinstance(controls.get("w_year"), W.SelectionRangeSlider):
    controls["w_year"].observe(render_all_widgets, "value")

# Rendu initial
render_all_widgets()
